# <br/><br/><br/>1. Anotacion de transcritos
# 2. Visualizacion de resultados en R<br/>
# <span style="color:red">!!! IMPORTANTE !!!</span>

## Requerimientos para la Ontologia <br/>
>### 1.- go.obo 
>### 2.- GO slim (goslim_generic)
>### 3.- Transcritos en formato fasta
>### 4.- Tener instalado BLAST
# <br/><br/>

In [ ]:
## Importamos modulos requeridos
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import shutil, os
from pandas import Series, DataFrame 
import pandas as pd
import pathlib
pd.set_option('max_rows',100000)
pd.set_option('max_colwidth',100000)
import urllib.request
import webbrowser
import re
import numpy as np
from time import sleep
import subprocess
from subprocess import Popen, PIPE, STDOUT
import shutil
from IPython.display import Image
from IPython.core.display import HTML
from datetime import datetime 

In [ ]:
os.makedirs('../data/Experimento2_GOslim_blast',exist_ok=True) 

# 1.- Descargas (go.obo y GO slim)<br/>

In [ ]:
if os.path.exists("../data/Experimento2_GOslim_blast//go.obo"): 
    print ('■■■ El archivo go.obo ya existe, no se descargará de la red')
    if os.path.exists("../data/Experimento2_GOslim_blast/all_go.txt"):
        print('■■■ El archivo all_go.txt ya existe')
    else:
        ## este aschivo sirve para generar un GAF nuevo
        output4=Popen("grep -e '^id: GO:' -e '^name: ' -e '^namespace: ' -e '.Typedef.' ../data/Experimento2_GOslim_blast/go.obo | sed 's/id:/#id:/g' | perl -p -e 's/\n/\t/g' | sed 's/.Typedef.*//g' | perl -p -e 's/#/\n/g' | sed 's/id: //g; s/name: //g; s/namespace: //g' | sed '/^$/d' | sed 's/\t$//g' | sed 's/biological_process/P/g; s/molecular_function/F/g; s/cellular_component/C/g' | grep -v -e '\tC\tC' -v -e '\tF\tF' -v -e '\tP\tP' > ../data/Experimento2_GOslim_blast//all_go.txt",
              shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True).stdout.read()
        print('■■■ Se ha creado el archivo all_go.txt')
else:
    ## descarga de la ontologia de genes
    output1=urllib.request.urlretrieve('http://snapshot.geneontology.org/ontology/go.obo','../data/Experimento2_GOslim_blast/go.obo')
    print('■■■ Se ha descargado el archivo go.obo')
    if os.path.exists("../data/Experimento2_GOslim_blast/all_go.txt"):
        print('■■■ El archivo all_go.txt ya existe')
    else:
        ## este aschivo sirve para generar un GAF nuevo
        output4=Popen("grep -e '^id: GO:' -e '^name: ' -e '^namespace: ' -e '.Typedef.' ../data/Experimento2_GOslim_blast/go.obo | sed 's/id:/#id:/g' | perl -p -e 's/\n/\t/g' | sed 's/.Typedef.*//g' | perl -p -e 's/#/\n/g' | sed 's/id: //g; s/name: //g; s/namespace: //g' | sed '/^$/d' | sed 's/\t$//g' | sed 's/biological_process/P/g; s/molecular_function/F/g; s/cellular_component/C/g' | grep -v -e '\tC\tC' -v -e '\tF\tF' -v -e '\tP\tP' > ../data/Experimento2_GOslim_blast/all_go.txt",
              shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True).stdout.read()        
        print('■■■ Se ha creado el archivo all_go.txt')
if os.path.exists("../data/Experimento2_GOslim_blast/goslim_generic.txt"):
    print ('■■■ El archivo goslim_generic.txt ya existe, no se descargará de la red')
else:
    ## descargamos el go-slim desde el Gene Ontology Consortium
    output3=urllib.request.urlretrieve('ftp://ftp.geneontology.org/go/GO_slims/goslim_generic.obo','../data/Experimento2_GOslim_blast/goslim_generic.obo')
    ## Ahora le hacemos una ligera edicion para obtener tres columnas: GO ID, término y aspecto
    print('■■■ Se ha descargado goslim_generic.obo')
    output4=Popen("grep -e '^id: GO:' -e '^name: ' -e '^namespace: ' -e '.Typedef.' ../data/Experimento2_GOslim_blast/goslim_generic.obo | sed 's/id:/#id:/g' | perl -p -e 's/\n/\t/g' | sed 's/.Typedef.*//g' | perl -p -e 's/#/\n/g' | sed 's/id: //g; s/name: //g; s/namespace: //g' | sed '/^$/d' | sed 's/\t$//g' | sed 's/biological_process/P/g; s/molecular_function/F/g; s/cellular_component/C/g' | grep -v -e '\tC\tC' -v -e '\tF\tF' -v -e '\tP\tP'  > ../data/Experimento2_GOslim_blast/goslim_generic.txt",
              shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True).stdout.read()
    print('■■■ Se ha creado el archivo goslim_generic.txt')
    if os.path.exists("../data/Experimento2_GOslim_blast/goslim_generic.obo"): os.remove("../data/Experimento2_GOslim_blast/goslim_generic.obo")

# 2.- Transcriptoma en estudio (fasta) 
>## Las secuencias de nucleotidos <span style="color:green">8_S356_contigs.fa</span> (fasta) se obtuvo despues de haber corrido el programa Trinity

In [ ]:
## Descarga del archivo fasta desde github
output=urllib.request.urlretrieve('https://raw.githubusercontent.com/eduardo1011/Programas/master/GO/8_S356_contigs.fa','../data/Experimento2_GOslim_blast/8_S356_contigs.fa')

# 3.- Blastx

In [ ]:
## Nombre de cada columna del resultado del blastx
header=("qacc","GENE PRODUCT ID","qlen","slen","length","score","bitscore","evalue","pident","nident",
                  "mismatch","positive","gaps","gapopen","stitle")

In [ ]:
## Ejecutamos el programa blastx (busca nucleotidos traducidos dentro en una base de datos de proteínas)
inicio = datetime.now()
#
!blastx -db ~/Desktop/bigdata/uniprot/uniprot.fasta \
-query ~/Desktop/data/fasta/8_S356_contigs.fa \
-evalue 1E-6 -outfmt '6 qacc sacc qlen slen length score bitscore evalue pident nident mismatch positive gaps gapopen stitle' \
-max_target_seqs 1 \
-max_hsps 1 \
-num_threads 6 \
-out ../data/Experimento2_GOslim_blast/blastx
#
lapso = datetime.now() - inicio
print('\nTiempo (hh:mm:ss.ms) {}'.format(lapso)+'\n')

In [ ]:
## Abrimos el resultado del blastx
blastx=pd.read_csv('../data/Experimento2_GOslim_blast/blastx',sep='\t',names=header)
print('\n',blastx[['qacc']].drop_duplicates().count().iloc[0],'transcritos')
print('\n',blastx[['GENE PRODUCT ID']].drop_duplicates().count().iloc[0],' proteinas')
blastx.head(3)

In [ ]:
### Se puede aplicar un filtro antes de seguir con el analisis
#proteínas con >80% de identidad
#
#blastx=blastx[(blastx.pident >= 80) & (blastx.pident <= 100)].reset_index(drop=True).drop_duplicates()
#print('\n',blastx[['GENE PRODUCT ID']].drop_duplicates().count().iloc[0],' proteinas con identidad mayor al 80%')
print('\n',blastx[(blastx.pident >= 80) & (blastx.pident <= 100)].reset_index(drop=True).drop_duplicates().count().iloc[0],' secuencias de proteinas con identidad mayor al 80%\n')
#blastx.head(3)

# 4.- Extraemos informacion funcional desde QuickGO
## Sintaxis del comando para extraer informacion desde QuickGO
## <span style="color:blue">" url + GO terms del goslim + IDs de productos de genes + salvar el archivo "</span><br/>

In [ ]:
## abrimos el GO slim especifico y lo asignamos
goslim_specific=pd.read_table('../data/Experimento2_GOslim_blast/goslim_generic.txt',names=['GO TERM','TERM','SLIM ASPECT'])
goslim_specific.head()

In [ ]:
## guardamos en una variable la ontologia completa
all_GO_terms=pd.read_table('../data/Experimento2_GOslim_blast/all_go.txt',names=['SLIMMED FROM','ALL TERM','GO ASPECT'])
all_GO_terms.head()

In [ ]:
## Conteo de GO terms de la ontologia completa
ont_completa=all_GO_terms.groupby('GO ASPECT')['SLIMMED FROM'].count().reset_index().sort_values(by ='SLIMMED FROM',ascending=False)
ont_completa= DataFrame(ont_completa)
ont_completa

In [ ]:
## Preparamos el goslim
## guardamos la salida de un comando de linux en una variable
goslim=Popen("cut -f1 ../data/Experimento2_GOslim_blast/goslim_generic.txt | perl -p -e 's/\n/\t/g' | sed 's/\t/,/g' | sed 's/,$//' ",
          shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True).stdout.read().decode()
print('\n',type(goslim))
print('\n',goslim,'\n')

In [ ]:
## Preparamos los IDs de las proteinas
## Extraemos el "ID subject" del blastx y creamos una cadena para extraer informacion de QuickGO
ids_blastx=Popen("cut -f2 ../data/Experimento2_GOslim_blast/blastx | sort | uniq | perl -p -e 's/\n/,UniProtKB:/g' | sed 's/,UniProtKB:$//' | sed 's/^/UniProtKB:/' ",
          shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True).stdout.read().decode()
print('\n',type(ids_blastx))
print('\n',ids_blastx,'\n')

In [ ]:
## Extraemos informacion funcional de los productos de genes desde QuickGO

## guardamos una variable constante (URL)
services_QuickGO = 'https://www.ebi.ac.uk/QuickGO/services/annotation/downloadSearch?goUsage=slim&goUsageRelationships=is_a,part_of,occurs_in&goId='
out = Popen("curl -X GET --header 'Accept:text/tsv' '"+services_QuickGO+goslim+"&geneProductId="+ids_blastx+"' > ../data/Experimento2_GOslim_blast/quickgo",
            shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True).stdout.read()

## Abrimos el archivo descargado extraemos las columnas: entry GO-id
go_quickgo=pd.read_csv('../data/Experimento2_GOslim_blast/quickgo',sep='\t')
print('\n',go_quickgo[['GENE PRODUCT ID']].drop_duplicates().count().iloc[0],' productos de genes con GO terms')
go_quickgo.head()

In [ ]:
## Descargamos un programa de perl para extraer informacion desde uniprot
output=urllib.request.urlretrieve('https://raw.githubusercontent.com/eduardo1011/Programas/master/Retrieve--information-id_from_IDUniprot.pl','../data/Experimento2_GOslim_blast/Retrieve--information-id_from_IDUniprot.pl')

go_quickgo[['GENE PRODUCT ID']].drop_duplicates().to_csv('../data/Experimento2_GOslim_blast/IDs.csv',index=None,header=None)

out3 = Popen("perl ../data/Experimento2_GOslim_blast/Retrieve--information-id_from_IDUniprot.pl ../data/Experimento2_GOslim_blast/IDs.csv > ../data/Experimento2_GOslim_blast/IDs_information_uniprot.tab",
             shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True).stdout.read()

orden_columnas=[0,1,2,3,4]
names={'Entry':'GENE PRODUCT ID','Protein names':'PROTEIN NAME','Gene names':'GENE NAME','Organism':'ORGANISM','Organism ID':'TAXON ID'}
informacion_uniprot=pd.read_csv('../data/Experimento2_GOslim_blast/IDs_information_uniprot.tab',
                                sep='\t',usecols=orden_columnas).rename(columns=names,index=str)
informacion_uniprot.head()

In [ ]:
## A los resultados del blast le agregamos informacion extraida de uniprot
quickgo_uniprot=pd.merge(go_quickgo,informacion_uniprot[['GENE PRODUCT ID','PROTEIN NAME','ORGANISM','TAXON ID']],on=['GENE PRODUCT ID','TAXON ID'],how='inner')
quickgo_uniprot.head(3)

In [ ]:
## a la tabla anterior le agregamos los terminos correspondientes a GOs especificos (SLIMMED FROM   ALL TERM)
quickgo_uniprot_allGO=pd.merge(quickgo_uniprot,all_GO_terms[['SLIMMED FROM','ALL TERM']],on='SLIMMED FROM',how='inner')
quickgo_uniprot_allGO.head(3)

In [ ]:
## a la tabla anterior le agregamos los terminos correspondientes al GO slim (GO TERM   TERM)
anotacion=pd.merge(quickgo_uniprot_allGO,goslim_specific[['GO TERM','TERM','SLIM ASPECT']],on='GO TERM',how='inner')
anotacion.head(4)

In [ ]:
## organismos identificados despues del blastx
org=anotacion[['GENE PRODUCT ID','ORGANISM','TAXON ID']].drop_duplicates().groupby(['TAXON ID','ORGANISM'])['GENE PRODUCT ID'].count().reset_index().sort_values(by ='GENE PRODUCT ID',ascending=False)
org= DataFrame(org)
org

In [ ]:
## productos de genes por aspecto, version slim
apecto_slim=anotacion[['GENE PRODUCT ID','SLIM ASPECT']].drop_duplicates().groupby('SLIM ASPECT')['GENE PRODUCT ID'].count().reset_index().sort_values(by ='GENE PRODUCT ID',ascending=False)
apecto_slim= DataFrame(apecto_slim)
apecto_slim.to_csv('../data/Experimento2_GOslim_blast/Total_categories_CFP.csv',header=None,index=None)
apecto_slim

In [ ]:
## productos de genes por aspecto, version completa
apecto_completo=anotacion.groupby('GO ASPECT')['GENE PRODUCT ID'].count().reset_index().sort_values(by ='GENE PRODUCT ID',ascending=False)
apecto_completo= DataFrame(apecto_completo)
apecto_completo

In [ ]:
## terminos por aspecto, version slim
x=anotacion[['TERM','SLIM ASPECT']].drop_duplicates().groupby('SLIM ASPECT')['TERM'].count().reset_index().sort_values(by ='TERM',ascending=False)
x= DataFrame(x)
x

In [ ]:
## productos de genes por GO term, version slim
ontologia_slim=anotacion[['GENE PRODUCT ID','GO TERM','TERM','SLIM ASPECT']].drop_duplicates().groupby(['SLIM ASPECT','GO TERM','TERM'])['GENE PRODUCT ID'].count().reset_index().sort_values(by ='SLIM ASPECT',ascending=False)
ontologia_slim= DataFrame(ontologia_slim)
ontologia_slim

In [ ]:
## productos de genes por GO term, version completa
ontologia_completa=anotacion[['GENE PRODUCT ID','SLIMMED FROM','ALL TERM',
                              'GO ASPECT']].drop_duplicates().groupby(['GO ASPECT',
                                                                       'SLIMMED FROM','ALL TERM'])['GENE PRODUCT ID'].count().reset_index().sort_values(by ='GENE PRODUCT ID',ascending=False)
ontologia_completa= DataFrame(ontologia_completa)
ontologia_completa

In [ ]:
## GO terms por termino, version slim
go_terms_slim=anotacion.groupby('TERM')['GO TERM'].count().reset_index().sort_values(by ='GO TERM',ascending=False).reset_index(drop=True)
go_terms_slim= DataFrame(go_terms_slim)
go_terms_slim

In [ ]:
## GO terms por termino, version completa
go_terms_completo=anotacion.groupby('ALL TERM')['SLIMMED FROM'].count().reset_index().sort_values(by ='SLIMMED FROM',ascending=False).reset_index(drop=True)
go_terms_completo= DataFrame(go_terms_completo)
go_terms_completo

In [ ]:
## fuentes de anotacion y total de productos de genes anotados
fuentes=anotacion[['GENE PRODUCT ID','ASSIGNED BY']].drop_duplicates().groupby('ASSIGNED BY')['GENE PRODUCT ID'].count().reset_index().sort_values(by ='GENE PRODUCT ID',ascending=False)
fuentes= DataFrame(fuentes)
fuentes

In [ ]:
## fuentes de anotacion y GO terms
fuentes_go=anotacion[['SLIMMED FROM','ASSIGNED BY']].drop_duplicates().groupby('ASSIGNED BY')['SLIMMED FROM'].count().reset_index().sort_values(by ='SLIMMED FROM',ascending=False)
fuentes_go= DataFrame(fuentes_go)
fuentes_go

# Resultados de la anotacion

## Funcion molecular

In [ ]:
anotacion_sindup=anotacion[['GENE PRODUCT ID','GO TERM','TERM','SLIM ASPECT']].drop_duplicates()

In [ ]:
## Molecular Function
func=anotacion_sindup.groupby(['SLIM ASPECT']).get_group('F')
function=func.groupby(['GO TERM','TERM']).count().reset_index().rename(columns={"GO TERM":"GO","TERM": "Molecular Function","SLIM ASPECT": "Value"}).sort_values(by ='Value',ascending=False).reset_index(drop=True)
function[['GO','Molecular Function','Value']].to_csv('../data/Experimento2_GOslim_blast/Molecular_Function.csv',index=None)
function
#mayor_A_10=function[(function.Value >= 1)]
#menor_A_10=function[(function.Value < 1)]
#otros_terminos={'GO':['GOs'],'Molecular Function':['Other Functions'],'GENE PRODUCT ID':[sum(menor_A_10.Value)],'Value':[sum(menor_A_10.Value)]}
#terminos=pd.DataFrame(otros_terminos)
#others=pd.concat([mayor_A_10,terminos[['GO','Molecular Function','GENE PRODUCT ID','Value']]],axis=0)
#others[['GO','Molecular Function','Value']].to_csv('../data/Experimento2_GOslim_blast/Molecular_Function.csv',index=None)
#others#.head()

In [ ]:
## matriz para ver intersecciones entre terminos
up_function=anotacion_sindup[anotacion_sindup['SLIM ASPECT'].str.contains("F")==True]
upset_function=up_function.pivot_table(values="TERM",index=["GENE PRODUCT ID"],aggfunc=len,columns="GO TERM").fillna('0').reset_index()
upset_function.to_csv('../data/Experimento2_GOslim_blast/Molecular_Function_matrix.csv',index=None)
upset_function

### Una forma de explorar y crear un grafico DAG con nuestros GO terms obtenidos
http://amigo.geneontology.org/amigo/software_list

In [ ]:
## guardamos la salida de un comando de linux en una variable
str_DAG=Popen("cut -d',' -f1 ../data/Experimento2_GOslim_blast/Molecular_Function.csv | sed '1d' | perl -p -e 's/\n/\t/g' | sed 's/\t/%0D%20/g' | sed 's/:/%3A/g' ",
          shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True).stdout.read().decode()
print('\n',type(str_DAG))
print('\n',str_DAG,'\n')
#url_visualizacion='http://amigo.geneontology.org/visualize?format=png&inline=false&mode=amigo&term_data_type=string&term_data='
url_visualizacion='http://amigo.geneontology.org/visualize?format=svg&mode=amigo&term_data='
#import webbrowser
#webbrowser.open(url_visualizacion+str_DAG, new=2)
print(url_visualizacion+str_DAG+'&inline=false&term_data_type=string')

## Proceso biologico

In [ ]:
## Biological Process
pros=anotacion_sindup.groupby(['SLIM ASPECT']).get_group('P')
process=pros.groupby(['GO TERM','TERM']).count().reset_index().rename(columns={"GO TERM":"GO","TERM": "Biological Process","SLIM ASPECT": "Value"}).sort_values(by ='Value',ascending=False).reset_index(drop=True)
process[['GO','Biological Process','Value']].to_csv('../data/Experimento2_GOslim_blast/Biological_Process.csv',index=None)
process
#mayor_A_10=process[(process.Value >= 1)]
#menor_A_10=process[(process.Value < 1)]
#otros_terminos={'GO':['GOs'],'Biological Process':['Other Processes'],'GENE PRODUCT ID':[sum(menor_A_10.Value)],'Value':[sum(menor_A_10.Value)]}
#terminos=pd.DataFrame(otros_terminos)
#others=pd.concat([mayor_A_10,terminos[['GO','Biological Process','GENE PRODUCT ID','Value']]],axis=0)
#others[['GO','Biological Process','Value']].to_csv('../data/Experimento2_GOslim_blast/Biological_Process.csv',index=None)
#others#.head()

In [ ]:
## matriz para ver intersecciones entre terminos
up_process=anotacion_sindup[anotacion_sindup['SLIM ASPECT'].str.contains("P")==True]
upset_process=up_process.pivot_table(values="TERM",index=["GENE PRODUCT ID"],aggfunc=len,columns="GO TERM").fillna('0').reset_index().rename(columns={"GENE PRODUCT ID":"Entry"})
upset_process.to_csv('../data/Experimento2_GOslim_blast/Biological_Process_matrix.csv',index=None)
upset_process.head()

# <span style="color:red">" Crea el grafico DAG como en Funcion molecular "</span><br/>

## Componente celular

In [ ]:
## Cellular Component
comp=anotacion_sindup.groupby(['SLIM ASPECT']).get_group('C')
component=comp.groupby(['GO TERM','TERM']).count().reset_index().rename(columns={"GO TERM":"GO","TERM": "Cellular Component","SLIM ASPECT": "Value"}).sort_values(by ='Value',ascending=False).reset_index(drop=True)
component[['GO','Cellular Component','Value']].to_csv('../data/Experimento2_GOslim_blast/Cellular_Component.csv',index=None)
component
#mayor_A_10=component[(component.Value >= 1)]
#menor_A_10=component[(component.Value < 1)]
#otros_terminos={'GO':['GOs'],'Cellular Component':['Other Components'],'GENE PRODUCT ID':[sum(menor_A_10.Value)],'Value':[sum(menor_A_10.Value)]}
#terminos=pd.DataFrame(otros_terminos)
#others=pd.concat([mayor_A_10,terminos[['GO','Cellular Component','GENE PRODUCT ID','Value']]],axis=0)
#others[['GO','Cellular Component','Value']].to_csv('../data/Experimento2_GOslim_blast/Cellular_Component.csv',index=None)
#others#.head()

In [ ]:
## matriz para ver intersecciones entre terminos
up_component=anotacion_sindup[anotacion_sindup['SLIM ASPECT'].str.contains("C")==True]
upset_component=up_component.pivot_table(values="TERM",index=["GENE PRODUCT ID"],aggfunc=len,columns="GO TERM").fillna('0').reset_index()
upset_component.to_csv('../data/Experimento2_GOslim_blast/Cellular_Component_matrix.csv',index=None)
upset_component.head()

# <span style="color:red">" Crea el grafico DAG como en Funcion molecular "</span><br/>

## 5.- Transcritos anotados y no anotados

In [ ]:
## Creamos un df con todos losidentificadores de la secuencia fasta
reads=Popen("grep  '>' ../data/fasta/8_S356_contigs.fa | sed 's/>//g' | cut -d' ' -f1 > ../data/Experimento2_GOslim_blast/reads.txt",
          shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT, close_fds=True).stdout.read().decode()
reads=pd.read_table('../data/Experimento2_GOslim_blast/reads.txt',header=None,names=['qacc'])
reads.head()

In [ ]:
## cuantos transcritos codifican a X proteina
trans=blastx.groupby('GENE PRODUCT ID')['qacc'].count().reset_index().sort_values(by ='qacc',ascending=False).reset_index(drop=True)
trans= DataFrame(trans)
trans

In [ ]:
print('Transcritos en fasta:     ',reads.drop_duplicates().count()[0])
print('Transcritos codificantes: ',blastx[['GENE PRODUCT ID']].count()[0])
print('Transcritos restantes:    ',reads.drop_duplicates().count()[0] - blastx[['GENE PRODUCT ID']].count()[0])

In [ ]:
## Esta tabla resume datos de las proteinas que fueron identificadas con el blastx
final=pd.merge(trans,anotacion[['GENE PRODUCT ID','SYMBOL','PROTEIN NAME','ORGANISM']],on='GENE PRODUCT ID',how='left' ).drop_duplicates().reset_index(drop=True)
final

In [ ]:
## visualizamos los transcritos que codifican para Q9NBX4
blastx[blastx['GENE PRODUCT ID'].str.contains('Q9NBX4')][['qacc']]

In [ ]:
## Archivo para generar un diagrama de Venn, intersecciones entre categorias, total de proteinas por categoria
ff=anotacion[anotacion['SLIM ASPECT'].str.contains("F")==True]
pp=anotacion[anotacion['SLIM ASPECT'].str.contains("P")==True]
cc=anotacion[anotacion['SLIM ASPECT'].str.contains("C")==True]
cats=[pp[['GENE PRODUCT ID']],ff[['GENE PRODUCT ID']],cc[['GENE PRODUCT ID']]]
categorias=pd.concat(cats, axis=1, ignore_index=True).rename(columns={0:"Process",1:'Function',2:'Component'},index=str).drop_duplicates().reset_index(drop=True)
categorias.to_csv('../data/Experimento2_GOslim_blast/cats.csv',index=None) 
categorias